# Import and inspect keras model

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [ ]:
model = load_model("./weights/keras_export")

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
# doesn't work: https://github.com/keras-team/keras/issues/11746
embedding = keras.Model(inputs=model.input, outputs=model.get_layer("activation_45"), name="embedding_model")

In [ ]:
# workaround: sequential api
sequential = keras.Sequential()
sequential.add(model.get_layer("input_1"))
sequential.add(model.get_layer("activation_45"))
sequential.compile()

In [ ]:
input = tf.random.uniform((1,112,8,8))
sequential.predict(input)

# Export keras model to onnx

In [ ]:
import tf2onnx

In [ ]:
spec = (tf.TensorSpec((None, 112, 8, 8), tf.float32, name="input"),)
output_path = model.name + ".onnx"
tf2onnx.convert.from_keras(model=model, opset=12, input_signature=spec, output_path=output_path)

In [ ]:
# https://github.com/microsoft/onnxruntime/issues/3229
import numpy as np
sess = onnxruntime.InferenceSession(output_path)
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
scores = sess.run([output_name], {input_name: np.random.random((1,112,8,8)).astype(np.float32)})[0]
scores